In [36]:
import requests
import pandas as pd
import re
import csv
from tqdm import tqdm

file_path = '/Users/michael/Documents/sounderpy/bufkit_urls.csv'
urls_df = pd.read_csv(file_path)

stations = []
latitudes = []
longitudes = []

for index, row in tqdm(urls_df.iterrows(), total=len(urls_df), desc="Processing URLs"):
    url = row['URL']
    response = requests.get(url)
    if response.status_code == 200:
        text = response.text

        stid_match = re.search(r"STID = (\w+)", text)
        slat_match = re.search(r"SLAT = ([-+]?\d+\.\d+)", text)
        slon_match = re.search(r"SLON = ([-+]?\d+\.\d+)", text)

        if stid_match and slat_match and slon_match:
            stid = stid_match.group(1)
            slat = slat_match.group(1)
            slon = slon_match.group(1)

            stations.append(stid)
            latitudes.append(slat)
            longitudes.append(slon)
    else:
        print(f"Failed to retrieve data from {url}")

data = {
    'Station': stations,
    'Latitude': latitudes,
    'Longitude': longitudes
}
output_df = pd.DataFrame(data)

output_file_path = '/Users/michael/Documents/sounderpy/stations_coordinates_scraped.csv'
output_df.to_csv(output_file_path, index=False)

print("Scraping completed and data saved to stations_coordinates_debug.csv")

Processing URLs: 100%|██████████| 1493/1493 [12:58<00:00,  1.92it/s]

Scraping completed and data saved to stations_coordinates_debug.csv


In [37]:
file_path = '/Users/michael/Documents/sounderpy/stations_coordinates_scraped.csv'
df = pd.read_csv(file_path)

valid_entries = []

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing entries"):
    if len(row['Station']) >= 3:
        valid_entries.append(row)

filtered_df = pd.DataFrame(valid_entries)

output_file_path = '/Users/michael/Documents/sounderpy/stations_coordinates_filtered.csv'
filtered_df.to_csv(output_file_path, index=False)

print("Filtering completed and data saved to stations_coordinates_filtered.csv")

Processing entries: 100%|██████████| 1493/1493 [00:00<00:00, 21698.94it/s]

Filtering completed and data saved to stations_coordinates_filtered.csv
